<h1 style="font-size:36pt"> Generate dashboard </h1>

Run entire notebook to generate the dashboard for **Local Climate Change Tool: Climate Change in your city**. 
Alternatively, you can run this from the command line:
```
panel serve climate_dashboard.ipynb --show
```
The dashboard is interactive with user input from the panel.widgets package and param classes. Report bugs to the collaborators [here](https://github.com/czarakas/local-climate-data-tool/issues). Your feedback is welcome.

In [ ]:
import hvplot.xarray
import numpy as np
import param
import panel as pn
import panel.widgets as pnw

import util_panel

## Define Global Variables

These variables are used throughout the notebook and increase the speed of the functions and classes so we don't have to read in the data more than once.

- THIS_EXPERIMENT_ID is used to read in the data
- EXPERIMENT_KEYS is used as keys for the data dictionary
- DICT_LATLON is a dictionary where the key is country names and the values are a list of dictionaries of city names that points to the latitude (-90 to 90) and longitude (-180 to 180) of that city
- COLORS are used for ploting in the order of the keys of DICT_SCENARIOS.
- DICT_SCENARIOS is a dictionary that links the user friendly scenario names with their shorter technical names.
- DICT_COLORS is a dictionary that links scenario names to colors for plotting
- SCENARIO is list of user-friendly-named scenarios

In [ ]:
# For reading in the data and used as keys to data dictionary
THIS_EXPERIMENT_ID = ['historical','ssp126', 'ssp370','ssp245','ssp585'] # For reading in zarr files
EXPERIMENT_KEYS = THIS_EXPERIMENT_ID.copy()
EXPERIMENT_KEYS.append('historical_obs') # For all data

# For end user, plotting, etc.
DICT_LATLON = util_panel.create_country2city2latlon_dict() # Courtesy of https://simplemaps.com/data/world-cities
COLORS = ['black','gray','blue','green','orange','red']
DICT_SCENARIOS = {'Modeled Historical':'historical', 'Observed Historical':'historical_obs',
                  'SSP1: Sustainability':'ssp126', 'SSP2: Middle-of-the-road':'ssp245',
                  'SSP3: Regional Rivalry':'ssp370', 'SSP5: Fossil Fueled Development':'ssp585'
                 }
DICT_COLORS = dict(zip(list(DICT_SCENARIOS.keys()), COLORS))
SCENARIO = list(DICT_SCENARIOS.keys())[2:]


# Read in the data with a function

There are three types of data to read in: 

1. Dummy data which was used to generate the layout for the dashboard but is no longer needed, 
2. Global data which is the global average of temperature for all the scenarios
3. Real data for each location on the globe

In [ ]:
DUMMY_DICT = util_panel.read_data('dummy')
GLOBAL_DICT = util_panel.read_data('global')
REAL_DICT = util_panel.read_data('real')

# Generate plots

Example used is Seattle to make sure the plots are doing what we expect, which is choosing a lat, lon and then plotting a timeseries of all the scenarios plus historical observations. The second plot is focused on one scenario such that we see the model uncertainties, by plotting the min, mean and max.

In [ ]:
# Used for examples: Seattle = 47.6062, 237.67
thislat = 47.6062
thislon = 237.67

In [ ]:
def hvplot(df, lat, lon, **kwargs):
    """Generates plot -- parameter of time_series function below"""
    colors=['black','blue','green','orange','red','gray']
    hv0 = df[0].hvplot(color=colors[0], label="Modeled Historical") 
    hv1 = df[1].hvplot(color=colors[1], label="SSP1: Sustainability")
    hv2 = df[2].hvplot(color=colors[2], label="SSP2: Middle-of-the-road") 
    hv3 = df[3].hvplot(color=colors[3], label="SSP3: Regional Rivalry") 
    hv4 = df[4].hvplot(color=colors[4], label="SSP5: Fossil Fueled Development") 
    hv5 = df[5].hvplot(color=colors[5], label="Observed Historical")
    ymax = df[4].max()+3
    ymin = df[0].min()-3
    return (hv0 * hv5 * hv1 * hv2 * hv3 * hv4).opts(width=1100, height=350, legend_position='top', 
                                                    ylabel='Temperature (C)', xlabel='Year',
                                                    fontsize={'labels': 16, 'xticks': 14, 'yticks': 14, 'legend':9.7}
                                                   )

def time_series(lat, lon, annual_mean=True, global_mean=False, view_fn=hvplot):
    """Generate interactive portions of longitude and latitude for data"""
    if global_mean:
        ds = GLOBAL_DICT
    else:
        ds = DUMMY_DICT
    df = [None] * 6
    for i, exp in enumerate(EXPERIMENT_KEYS):
        one_ds = ds[exp]
        data_to_plot = one_ds.sel(lat=lat, lon=lon, method='nearest')
        if annual_mean:
            df[i] = data_to_plot.groupby('time.year').mean()['mean']
        else:
            df[i] = data_to_plot['mean']
    return view_fn(df, lat, lon)

# Test that above generates plot
# time_series(thislat, thislon, annual_mean=True, global_mean=False)

In [ ]:
def hvplot1(df, df_hist, scenario, lat, lon, **kwargs):
    """Generates plot -- parameter of time_series function below"""
    colors = ['black','blue','green','orange','red','gray']
    hv1 = df['mean'].hvplot.line(color=DICT_COLORS[scenario], label=scenario)
    hv2 = df[['min','max']].hvplot.area(y='min', y2='max', color=DICT_COLORS[scenario], alpha=0.5, legend=False)
    hv3 = df_hist['mean'].hvplot.line(color='black', label='Modeled Historical')
    hv4 = df_hist[['min','max']].hvplot.area(y='min', y2='max', alpha=0.5, 
                                             color=DICT_COLORS['Modeled Historical'], legend=False
                                            )
    ymax = df['max'].max() + 3
    ymin = df_hist['min'].min() - 3
    return (hv2 * hv4 * hv3 * hv1).opts(width=1100, height=400, ylabel='Temperature (C)', xlabel='Year',
                                        title="Scenario (%s) and Historical model range"%(scenario),
                                        fontsize={'title': 18, 'labels': 16, 'xticks': 14, 'yticks': 14, 'legend':9.7},
                                        legend_position='top'
                                        )

def time_series1(scenario, lat, lon, annual_mean=True, global_mean=False, view_fn=hvplot1):
    """Generate interactive portions of longitude and latitude for data"""
    ds = DUMMY_DICT
    one_ds = ds[DICT_SCENARIOS[scenario]]
    data_to_plot = one_ds.sel(lat=lat, lon=lon, method='nearest')
    dfh = ds['historical'].sel(lat=lat, lon=lon, method='nearest')
    if annual_mean:
        df = data_to_plot.groupby('time.year').mean()
        df_hist = dfh.groupby('time.year').mean()
    else:
        df = data_to_plot
        df_hist = dfh
    return view_fn(df, df_hist, scenario, lat, lon)

# Test that above generates plot
# time_series1('SSP5: Fossil Fueled Development', thislat, thislon, annual_mean=True, global_mean=False)

## Generate Widgets

This portion of the code generates many widgets and classes of parameters that depend on each other and update the plots.

- refresh: pnw.Button to load all changes to parameters at once. Plot will not update unless this button is pressed.
- latitude: pnw.FloatSlider to customize user input location
- longitude: pnw.FloatSlider to customize user input location
- scenario: pnw.Select to customize the second plot to show model uncertainties within the user selected scenario
- text: pnw.TextInput to display the currently selected city or lat/lon
- annual_mean: pnw.Checkbox - check for annual mean (much cleaner plot), uncheck to see annual variation (aka seasonal temperature change), default to True

Then, we created three classes.
1. Select: update city list from chosen country
2. Plots: generate first plot with user input location based on city or custom lat/lon
3. PlotScenario: genearte second plot with chosen scenario (same location as first plot)

Finally, we made a dependency on the refresh button that calls function *b* everytime the refresh button is clicked. This initiates a new rendering of the plots with data from new location, scenario, or annual mean.


In [ ]:
pn.extension()
background = 'WhiteSmoke'
refresh = pnw.Button(name='Refresh plots', button_type='primary')
# Custom Location tab
latitude = pnw.FloatSlider(name='latitude', value=0, start=-80, end=80)
longitude = pnw.FloatSlider(name='longitude', value=180, start=0, end=360)
# Scenario
scenario = pnw.Select(name='Future scenario', options=SCENARIO)
# Text to display currently selected location
text = pnw.TextInput(value='Select your location then press Refresh plots')
annual_mean = pnw.Checkbox(name = 'Annual Mean')
annual_mean.value = True #initialize annual_mean to be True

class Select(param.Parameterized):
    """Builds a timeseries plot of scenarios based on user input"""
    countries = sorted(list(DICT_LATLON.keys()))
    countries.insert(0,'Global')
    country = param.ObjectSelector(default='United States', objects=countries)
    city = param.ObjectSelector(default='Seattle', objects=sorted(list(DICT_LATLON['United States'].keys())))
    
    @param.depends('country', watch=True)
    def _update_cities(self):
        """Updates city list when country changes"""
        if self.country == 'Global':
            self.param['city'].objects = ['']
            self.city = ''
        else:
            cities = sorted(list(DICT_LATLON[self.country].keys()))
            self.param['city'].objects = cities
            self.city = cities[0]

select_city = Select()
tabs = pn.Tabs(('Choose City', pn.Row(select_city.param)), ('Custom Location', pn.Column(latitude, longitude)))

class Plots(param.Parameterized):
    """Builds timeseries for specific location"""
    ann_mean = param.Boolean(True, doc="check for plot of annual mean")
    update = param.Boolean(True)
    global_mean = param.Boolean(False)
    lat = param.Number(thislat, bounds=(-90, 90))
    lon = param.Number(thislon, bounds=(0, 360))

    @param.depends('update')
    def view(self):
        """Builds the plot using time_series function"""
        return time_series(self.lat, self.lon, annual_mean=self.ann_mean, global_mean=self.global_mean)

class PlotScenario(param.Parameterized):
    """Builds timeseries that dynamically updates"""
    lat = param.Number(thislat, bounds=(-90,90))
    lon = param.Number(thislon, bounds=(0, 360))
    global_mean = param.Boolean(False)
    update = param.Boolean(True)
    sce = param.ObjectSelector(default=SCENARIO[0], objects=SCENARIO)
    
    @param.depends('update')
    def view(self):
        """Builds a plot using time_series1 function"""
        return time_series1(self.sce, self.lat, self.lon, annual_mean=True, global_mean=self.global_mean)
    
plots = Plots()
plot_scenario = PlotScenario()

def b(event):
    if tabs.active == 0:
        # Select Country/City
        text.value ='Location Selected: {0}, {1}'.format(select_city.city, select_city.country)
        pos = DICT_LATLON[select_city.country][select_city.city]
        # Update plot of timeseries
        plots.lat = pos[0]
        if pos[1] < 0:
            new_lon = 360 + pos[1]
        else: 
            new_lon = pos[1]
        plots.lon = new_lon
        plots.ann_mean = annual_mean.value
        plots.update = not(plots.update)
    else:
        # Select Custom Lat/Lon
        text.value = 'Location Selected: Lat {0}°, Lon {1}°'.format(round(latitude.value,2), round(longitude.value,2))
        # Update plot of timeseries
        plots.lat = latitude.value
        plots.lon = longitude.value
        plots.ann_mean = annual_mean.value
        plots.update = not(plots.update)
    # Update plot of selected scenario
    plot_scenario.lat = plots.lat
    plot_scenario.lon = plots.lon
    plot_scenario.sce = scenario.value
    plot_scenario.update = not(plot_scenario.update)

refresh.param.watch(b, 'clicks');

widgets = pn.Column(tabs, annual_mean, scenario, refresh, text, background=background)

### Test that plots parameters update with widgets
Aka make sure that interactivity portion does what its supposed to do

Uncomment the cell below play with the widgets yourself and see how the parameters inside the plot classes are updated. Shows that the data being plotted is actually at the location we want.

In [ ]:
# pn.Row(widgets, plots.param, plot_scenario.param)

### Put all the widgets and plots together

The meat and potatos of our dashboard

In [ ]:
board = pn.Row(widgets, pn.Column(plots.view, plot_scenario.view))

# Generate the rest of the webpage content
- header (explainng how to use the widgets and update the plots)
- body text (explaining components of plots and where the data comes from)

### Generate header text
Describe how to use widgets to update plots and direct to more information

In [ ]:
text_header = '''
<h1 style="font-size:38pt;color:#224CA8;font-family:Gill Sans MT"> Local Climate Change Tool </h1> <p style="font-size:18pt;color:#224CA8;font-family:Gill Sans MT"> Climate Change in your City \n -------------- \n </p>
<p>
From the lists below, select a tab for choosing a location based on a dropdown list of cities or customize your latitude and longitude under <i> Custom Location </i> tab. Note that longitude is from 0 to 360. To refresh the plot with your new values hit the blue <i>Refresh Plots</i> button. For more information on these plots and where the data came from scroll down the page, or see <a href="https://github.com/czarakas/local-climate-data-tool">here</a>.
</p>
'''

header = pn.pane.Markdown(text_header, width=750, height=420, style={'font-family':'Arial','font-size':'14pt'})

### Generate body text
Describe elements of the plot, where the data came from, and what SSP means

In [ ]:
text_for_panel = '''
# <p style="font-family:Gill Sans MT;font-size:24pt"> Description of Local Climate Change Data Tool elements </p>
## <p style="font-size:18pt"> Plots </p>
<p> These plots show the near-surface air temperature for the grid point closest to your selected location from the CMIP6 climate model projections (1.125°x1.125° latitude-longitude grid) for the historical experiment and for each of the four scenarios. The mean, minimum, and maximum are computed over 16 models for the historical experiment, 11 models for SSP1: Sustainability, 10 models for SSP3: Regional Rivalry, and 9 models for SSP5: Fossil Fueled Development. The historical experiment was run for 165 years from 1850-2014, while the scenarios were run for 85 years from 2015-2100. This data is plotted monthly, so you will see a lot of noise from the seasonal temperature changes unless annual mean is selected. In the bottom plot, the model data is plotted alongside historical observations of surface temperature.
</p>
## <p style="font-family:Gill Sans MT;font-size:18pt"> Uncertainty </p>
<p style="font-size:14pt"> Note that uncertainty inherently arises in these simulations from the models themselves, from factors in the scenarios (such as emission), and from the natural internal variability in the climate. Uncertainty from the models is demonstrated in the spread between the minimum and maximum values for each scenario.
</p> '''
text_for_panel2=''' 
<h2 style="font-family:Gill Sans MT;font-size:18pt"> CMIP6 </h2>
<p> The Coupled Model Intercomparison Project, Phase 6 (CMIP6) is an intercomparison of over 30 global climate models with the same forcings and with a consistent output format to facilitate comparison. These models were run for a historical experiment to recreate the past to validate the models and for several different future scenarios. Currently, this tool only utilizes a subset of these climate models; this will be updated as more data becomes available. For more information, visit [https://www.wcrp-climate.org/wgcm-cmip](https://www.wcrp-climate.org/wgcm-cmip).
</p>

<h2 style="font-family:Gill Sans MT;font-size:18pt"> SSPs </h2>
<p> These scenarios are different shared socioeconomic pathways (SSPs), which make projections on what would happen in the future if that amount of radiative forcing is present, assuming no further climate change or changing policies. Each scenario is named after the amount of radiative forcing from human activities (e.g. the amount of carbon-based energy used and carbon emissions), meaning the amount of excess solar radiation entering the earth’s atmosphere. In the order of most to least sustainable, these radiative forcings are 1-2.6 W/m2 (SSP1: Sustainability), 2-4.5 W/m2 (SSP2: Middle-of-the-road), 3-7 W/m2 (SSP3: Regional Rivalry), and 5-8.5 W/m2 (SSP5: Fossil Fueled Development). Generally, higher forcing corresponds to warmer temperatures on Earth.
</p>

<h2 style="font-family:Gill Sans MT;font-size:18pt"> Observations </h2>
<p> <br clear=right/> <a href="http://berkeleyearth.org/"> <img src="http://static.berkeleyearth.org/img/berkeley-earth-logo-1.png" width=120 height=60 align='left'/></a> These temperature observations come from the Berkeley Earth Surface Temperature (BEST) dataset, which consists of monthly means of land surface air temperature observations that have been structured onto a 1° x 1° latitude-longitude grid, although observations may not be available for every point on the grid at all time steps. For more information, visit <a href="http://berkeleyearth.org/"> http://berkeleyearth.org/</a>.
</p>
'''
text_description = pn.Column(pn.pane.Markdown(text_for_panel, width=1380, style={'font-family': "Arial",'font-size':'14pt'}), 
                             pn.panel("bokeh_plot_uncertainty.gif", width=800), 
                             pn.pane.Markdown(text_for_panel2, width=1380, style={'font-family': "Arial",'font-size':'14pt'}))

# Put it all together! 
Generate full dashboard here. Uncomment 
```
 climate_tool.show()
```
in the cell below to launch in a new tab of your browser. 

In [ ]:
climate_tool = pn.Column(header, board, text_description)
climate_tool.servable();
# climate_tool.show()